# Hello!
Thanks for taking the time to use this uniform assignment tool.

There's a few of these informational blocks at the beginning of the notebook to help guide you, but the more detailed setup guide can be found here: [Click Me!](https://washingtonyouthsoccerorg-my.sharepoint.com/:p:/g/personal/thomask_washingtonyouthsoccer_org/EbvPZLljEJxAnjKTaPjoNfAB6srge8L8mweC7YJs2vYn_Q?e=PaorOX)

Read that guide in its entirety before beginning to run any cells of code in this window.

Below, you will find a lot of blocks of code, broken into little bits.

If you run them all in order, and you've gotten all the files properly formatted, then this will spit out an excel spreadsheet detailing which uniform each player should have.





# Preparatory Steps
* Ensure that your files are formatted as either team files or uniform files
* Double-check to make sure that you've saved your files as .csv files - the code has a hard time reading regular excel files.
* Ensure that you've gotten your files loaded into the folder labeled 'content' on the side panel to the left of this message.

# Instructions for running a code cell
* Click on a cell to highlight it
* Once highlighted, press 'Ctrl + Enter' to run the cell
* Pressing 'Shift + Enter' instead will run the cell & automatically highlight the next cell
* For this notebook, you should be able to press 'Shift + Enter', starting from the top cell, until all code cells have been run


# Other Things

Most of the code cells don't provide any kind of output. You can tell if a code cell was successfully executed if a tiny green checkmark appears near the upper-left corner of the cell after running it.

If you encounter any issues, refer to the accompanying powerpoint slides. If the issue cannot be resolved, contact ThomasK@washingtonyouthsoccer.org

In [ ]:
pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import required resources
import os
import json
import numpy as np
import pandas as pd
import unittest
import math

In [ ]:
# Accepts a filepath to a directory
# Scans through all files in the directory, returning only files
def filescan(path):
  files = []
  for f in os.scandir(path):
    if not f.is_dir():
      files.append(f)
  return files

In [ ]:
# Reads in a DataFrame and converts sizes to decimal representations
# Returns a new DataFrame
def str_to_deci(items):
  size_dict = {
    'YS': 1.0,
    'YM': 2.0,
    'YL': 3.0,
    'YXL': 3.75,
    'AXS': 3.25,
    'AS': 4.0,
    'AM': 5.0,
    'AL': 6.0,
    'AXL': 7.0,
    'A2XL': 8.0
  }
  items = items.replace({'Size': size_dict})
  return items

In [ ]:
def deci_to_str(item):
    size_dict = {
    '1.0': 'YS',
    '2.0': 'YM',
    '3.0': 'YL',
    '3.75': 'YXL',
    '3.25': 'AXS',
    '4.0': 'AS',
    '5.0': 'AM',
    '6.0': 'AL',
    '7.0': 'AXL',
    '8.0': 'A2XL'
    }
    return size_dict[str(float(item))]

In [ ]:
def formdict(filepath, attr):
  i_dict = {}
  for bin in filescan(filepath):
    content = str_to_deci(pd.read_csv(bin))
    attribute = content[attr][0]
    if attribute in i_dict:
      i_dict[attribute] = pd.concat([i_dict[attribute], content])
    else:
      i_dict[attribute] = content
  return i_dict

In [ ]:
# MMS = mismatch score
def mms(sdf, tdf):
  teamdict = {}
  team = []
  unis = []
  mismatch_score = 0
  tolerance = 0
  rdf = tdf.copy()
  udf = sdf.copy()

  while not rdf.empty and tolerance <= 3:
    for uniform in udf.iterrows():
      for player in rdf.iterrows():
        diff = abs(uniform[1][1] - player[1][1])
        if diff <= tolerance and not (player[1][0] in team or uniform[1][0] in unis):
          mismatch_score += abs(diff)
          team.append(player[1][0])
          unis.append(uniform[1][0])
          teamdict[player[1][0]] = (uniform[1][0], uniform[1][1], uniform[1][2])
      rdf = rdf[~rdf['Name'].isin(team)]
      team = []
    udf = udf[~udf['Number'].isin(unis)]
    tolerance += 0.25
  if not rdf.empty:
    mismatch_score = 999
  return mismatch_score, teamdict

In [ ]:
def qtyorder(sdf):
  sizecounts = sdf.value_counts(subset='Size', sort=True, ascending=True)
  limdf = sdf.copy().iloc[0:0]
  for size in sizecounts.index:
    tdf = sdf.loc[sdf['Size'] == size]
    limdf = pd.concat([limdf, tdf])
  return limdf

In [ ]:
def permute(inv, teamdict):
  mismatches = pd.DataFrame(columns=["Team Mismatch Score", "Team", "Style", "Breakdown"])
  for style in inv:
    styledf = inv[style]
    limdf = qtyorder(styledf)
    for team in teamdict:
      teamdf = teamdict[team]
      if limdf.shape[0] > teamdf.shape[0]:
        mis, breakdown = mms(limdf, teamdf)
        mismatches.loc[len(mismatches.index)] = {"Team Mismatch Score": mis, "Team": team, "Style": style, "Breakdown": breakdown}
      else:
        breakdown = {}
        mismatches.loc[len(mismatches.index)] = {"Team Mismatch Score": 999, "Team": team, "Style": style, "Breakdown": breakdown}
  return mismatches

In [ ]:
def claimUnis(style, p_dict):
  for player in p_dict: # For every player in this dictionary
    p_info = p_dict[player] # Grab the uniform size and number
    style = style.query("Number != @p_info[0] or Size != @p_info[1]")
  return style

In [ ]:
def assignmentCycle(sd, td):
  # Needs to be repeated until all teams are assigned uniforms
  mismatches = permute(sd, td).reset_index()
  chosen = mismatches.loc[mismatches['Team Mismatch Score'].idxmin()]
  playerdict = chosen['Breakdown']
  del td[chosen['Team']]
  # Updates the dictionary of data for the chosen style, removing those uniforms from the pool
  sd[chosen['Style']] = claimUnis(sd[chosen['Style']], playerdict)
  return chosen

In [ ]:
def writesheet(sheetpath, teamdata):
  with pd.ExcelWriter(sheetpath, mode='w') as writer:
    for team in teamdata:
      data = pd.DataFrame(team)
      for entrydict in data.loc['Breakdown']:
        tdf = pd.DataFrame(entrydict).T
        if not tdf.empty:
          tdf[1] = tdf.iloc[:][1].map(lambda x: deci_to_str(x)).values
          tdf.rename(columns={0: "Uniform Number", 1: 'Uniform Size', 2: 'Style'}, inplace=True)
        tdf.to_excel(writer, sheet_name=team.loc['Team'])


In [ ]:
# These only ever run once, to initialize the variables.
final_nums = {}
sizedf = {}
path = os.getcwd()
inventory = path + '/Uniforms'
rosters = path + '/Teams'
styledict = formdict(inventory, 'Style')
teamdict = formdict(rosters, 'Team')
final_teams = []

# Hi again!

The below code cell takes a while to run, about 10-20 seconds per team. It will continually produce output to let you know that it's still running.

In [ ]:
# Main method that calls all the others to form a final roster
# This one takes a while - it will periodically print which cycle it's on. There should be 1 cycle per team.
cycle = 1
while bool(teamdict):
  print("Cycle:", cycle)
  cycle += 1
  final_teams.append(assignmentCycle(styledict, teamdict))
  print('Mismatch Score:', final_teams[-1]['Team Mismatch Score'], final_teams[-1]['Team'])
print('Done Running!')

Cycle: 1
Mismatch Score: 0.0 Boys 2012 White
Cycle: 2
Mismatch Score: 0.0 Girls 2011 White
Cycle: 3
Mismatch Score: 0.0 Girls 2011 Green
Cycle: 4
Mismatch Score: 0.0 Boys 2011 Blue
Cycle: 5
Mismatch Score: 0.0 Boys 2012 Blue
Cycle: 6
Mismatch Score: 0.0 Boys 2006
Cycle: 7
Mismatch Score: 0.0 Girls 2006
Cycle: 8
Mismatch Score: 0.0 Girls 2008
Cycle: 9
Mismatch Score: 0.75 Girls 2010 White
Cycle: 10
Mismatch Score: 1.5 Girls 2010 Green
Cycle: 11
Mismatch Score: 2.25 Girls 2009
Cycle: 12
Mismatch Score: 4.25 Boys 2011 Green
Cycle: 13
Mismatch Score: 8.5 Boys 2010 Green
Cycle: 14
Mismatch Score: 20.25 Boys 2011 White
Cycle: 15
Mismatch Score: 999 Boys 2009
Cycle: 16
Mismatch Score: 999 Boys 2008
Cycle: 17
Mismatch Score: 999 Boys 2007
Cycle: 18
Mismatch Score: 999 Girls 2012 Blue
Cycle: 19
Mismatch Score: 999 Girls 2012 White
Cycle: 20
Mismatch Score: 999 Girls 2012 Green
Cycle: 21
Mismatch Score: 999 Girls 2007
Cycle: 22
Mismatch Score: 999 Boys 2012 Green
Done Running!


In [ ]:
writesheet(path + '/Assignment.xlsx', final_teams)

# That's it!
If you collapse & reopen the content folder, there should now be an excel document there titled Assignment.xlsx

Download that file, open it, and it will contain all the uniform assignment information for all teams that could have been outfitted.

Any empty tabs are the result of teams that could not be fitted with the current stock of uniforms - either because the fit was so bad (e.g. AXL -> AS), or because there were not enough uniforms of a single cohesive style to outfit the entire team (e.g. 14 uniforms of Primegreen style -/-> 18 player roster)